In [2]:
!pip install -q scikit-learn numpy python-levenshtein gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 73.2 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import Levenshtein  # (pip install python-levenshtein)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
import warnings

# 경고 무시
warnings.filterwarnings('ignore')

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:

# 2-1편의 도구를 재사용
stop_words_set = set(stopwords.words('english'))

# --- 2. 범용 전처리 함수 ---
def preprocess_to_tokens(text, stop_words):
  """전처리: 토큰화, 소문자화, 불용어/구두점 제거 후 '토큰 리스트' 반환"""
  tokens = word_tokenize(text.lower())
  filtered_tokens = [
    word for word in tokens
    if word.isalpha() and word not in stop_words
  ]
  return filtered_tokens

In [6]:


# --- 3. 유사도 계산기 구현 ---

## 3-1. 집합 기반 (Set-based) 유사도
# (단어의 공유 여부에만 관심)

def jaccard_similarity(text1, text2, stop_words):
  """자카드 유사도: (교집합 크기) / (합집합 크기)"""
  set1 = set(preprocess_to_tokens(text1, stop_words))
  set2 = set(preprocess_to_tokens(text2, stop_words))

  intersection = set1.intersection(set2)
  union = set1.union(set2)

  if not union:
    return 0.0
  return len(intersection) / len(union)

def dice_coefficient(text1, text2, stop_words):
  """다이스 계수: 2 * (교집합 크기) / (set1 크기 + set2 크기)"""
  set1 = set(preprocess_to_tokens(text1, stop_words))
  set2 = set(preprocess_to_tokens(text2, stop_words))

  intersection = set1.intersection(set2)

  if not set1 and not set2:
    return 0.0
  return (2 * len(intersection)) / (len(set1) + len(set2))

## 3-2. 편집 거리 기반 (Edit-based) 유사도
# (문자열 자체의 모양에만 관심, 오타/수정에 사용)

def levenshtein_similarity(text1, text2):
  """레벤슈타인 유사도: (1 - 편집 거리 / 최대 길이)"""
  # 전처리 없이 원본 텍스트(소문자)를 비교
  s1 = text1.lower()
  s2 = text2.lower()

  # 거리(Distance) 계산 (0이면 동일)
  distance = Levenshtein.distance(s1, s2)

  # 유사도(Similarity)로 변환 (1이면 동일)
  max_len = max(len(s1), len(s2))
  if max_len == 0:
    return 1.0

  return 1.0 - (distance / max_len)


## 3-3. 벡터 기반 (Vector-based) 유사도
# (단어의 의미/중요도에 관심)

# 가. TF-IDF + 코사인 유사도
tfidf_vectorizer = TfidfVectorizer()

def tfidf_cosine_similarity(text1, text2, stop_words):
  """TF-IDF 벡터 간의 코사인 유사도"""
  # TF-IDF는 문맥(corpus)이 있어야 하지만,
  # 여기서는 두 문장만으로 간단히 벡터화합니다.

  # 전처리된 '문장' (토큰 리스트가 아님)이 필요
  text1_processed = " ".join(preprocess_to_tokens(text1, stop_words))
  text2_processed = " ".join(preprocess_to_tokens(text2, stop_words))

  try:
    tfidf_matrix = tfidf_vectorizer.fit_transform([text1_processed, text2_processed])
    return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
  except ValueError:
    # (예: 두 문장 모두 불용어로만 구성되어 어휘가 0개일 때)
    return 0.0


# 나. (1편 응용) GloVe 임베딩 + 코사인 유사도
def load_glove_model():
  """사전 학습된 GloVe 모델 로드"""
  try:
    # gensim.downloader를 api로 임포트 (정상 작동)
    glove_model = api.load("glove-wiki-gigaword-50")
    print("\n(GloVe-wiki-gigaword-50 모델 로드 완료)")
    return glove_model
  except Exception as e:
    print(f"\n모델 로드 실패: {e}")
    print("인터넷 연결을 확인하거나 'glove-wiki-gigaword-50' 모델을 수동으로 다운로드하세요.")
    return None

# (이전 코드에서 이미 로드 시도했으므로, 다시 로드)
glove_model = load_glove_model()

[==================================================] 100.0% 66.0/66.0MB downloaded

(GloVe-wiki-gigaword-50 모델 로드 완료)


In [7]:
def get_sentence_vector(text, model, stop_words):
  """문장을 단어 벡터의 '평균'으로 변환 (Sentence Embedding)"""
  tokens = preprocess_to_tokens(text, stop_words)
  vectors = []

  for word in tokens:
    if word in model: # 모델 어휘 사전에 단어가 있는지 확인
      vectors.append(model[word])

  if not vectors:
    return np.zeros(model.vector_size)

  #
  # 모든 유효한 단어 벡터의 평균을 내어 '문장 벡터' 생성
  return np.mean(vectors, axis=0)


def glove_cosine_similarity(text1, text2, model, stop_words):
  """GloVe 임베딩 벡터 간의 코사인 유사도"""
  if model is None:
    return 0.0

  v1 = get_sentence_vector(text1, model, stop_words)
  v2 = get_sentence_vector(text2, model, stop_words)

  # sklearn은 2D 배열 입력을 요구하므로 reshape
  v1 = v1.reshape(1, -1)
  v2 = v2.reshape(1, -1)

  #
  return cosine_similarity(v1, v2)[0][0]


In [8]:

# --- 4. "응용 프로그램" 테스트 ---

# 테스트할 문장 쌍
s1 = "The king rules the country"
s2 = "The queen governs the nation"
s3 = "I love this great movie"
s4 = "I hate this terrible film"
s5 = "How old are you?"
s6 = "What is your age?"

print("\n--- [비교 1: 의미는 비슷하나 단어가 다름] ---")
print(f"  S1: {s1}")
print(f"  S2: {s2}")
print(f"  > Jaccard (단어 공유):     {jaccard_similarity(s1, s2, stop_words_set):.4f}")
print(f"  > TF-IDF+Cosine (중요도):  {tfidf_cosine_similarity(s1, s2, stop_words_set):.4f}")
print(f"  > GloVe+Cosine (의미):    {glove_cosine_similarity(s1, s2, glove_model, stop_words_set):.4f}")
print(f"  > Levenshtein (문자열):   {levenshtein_similarity(s1, s2):.4f}")


print("\n--- [비교 2: 구조는 비슷하나 의미가 반대] ---")
print(f"  S3: {s3}")
print(f"  S4: {s4}")
print(f"  > Jaccard (단어 공유):     {jaccard_similarity(s3, s4, stop_words_set):.4f}")
print(f"  > TF-IDF+Cosine (중요도):  {tfidf_cosine_similarity(s3, s4, stop_words_set):.4f}")
print(f"  > GloVe+Cosine (의미):    {glove_cosine_similarity(s3, s4, glove_model, stop_words_set):.4f}")
print(f"  > Levenshtein (문자열):   {levenshtein_similarity(s3, s4):.4f}")


print("\n--- [비교 3: 의미는 같으나 표현이 다름] ---")
print(f"  S5: {s5}")
print(f"  S6: {s6}")
print(f"  > Jaccard (단어 공유):     {jaccard_similarity(s5, s6, stop_words_set):.4f}")
print(f"  > TF-IDF+Cosine (중요도):  {tfidf_cosine_similarity(s5, s6, stop_words_set):.4f}")
print(f"  > GloVe+Cosine (의미):    {glove_cosine_similarity(s5, s6, glove_model, stop_words_set):.4f}")
print(f"  > Levenshtein (문자열):   {levenshtein_similarity(s5, s6):.4f}")


--- [비교 1: 의미는 비슷하나 단어가 다름] ---
  S1: The king rules the country
  S2: The queen governs the nation
  > Jaccard (단어 공유):     0.0000
  > TF-IDF+Cosine (중요도):  0.0000
  > GloVe+Cosine (의미):    0.8530
  > Levenshtein (문자열):   0.3929

--- [비교 2: 구조는 비슷하나 의미가 반대] ---
  S3: I love this great movie
  S4: I hate this terrible film
  > Jaccard (단어 공유):     0.0000
  > TF-IDF+Cosine (중요도):  0.0000
  > GloVe+Cosine (의미):    0.8566
  > Levenshtein (문자열):   0.4400

--- [비교 3: 의미는 같으나 표현이 다름] ---
  S5: How old are you?
  S6: What is your age?
  > Jaccard (단어 공유):     0.0000
  > TF-IDF+Cosine (중요도):  0.0000
  > GloVe+Cosine (의미):    0.6041
  > Levenshtein (문자열):   0.2353
